In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
    # !pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [7]:
import json
from unsloth import FastVisionModel
from datasets import load_dataset
from tqdm import tqdm
import torch

model, processor = FastVisionModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit = True,
)
FastVisionModel.for_inference(model)
# The model all the padding on the left side so we can extract the last token for sentence summary.
processor.tokenizer.padding_side = "left"

==((====))==  Unsloth 2025.11.6: Fast Mllama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MllamaForConditionalGeneration(
  (model): MllamaModel(
    (vision_model): MllamaVisionModel(
      (patch_embedding): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), padding=valid, bias=False)
      (gated_positional_embedding): MllamaPrecomputedPositionEmbedding(
        (tile_embedding): Embedding(9, 8197120)
      )
      (pre_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
        (embedding): Embedding(9, 5120)
      )
      (post_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
        (embedding): Embedding(9, 5120)
      )
      (layernorm_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (layernorm_post): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (transformer): MllamaVisionEncoder(
        (layers): ModuleList(
          (0-12): 13 x MllamaVisionEncoderLayer(
            (self_attn): MllamaVisionAttention(
              (q_proj): Linear4bit(in_features=1280, out_features=1280, bias=False)
       

In [4]:
from datasets import load_dataset
dataset = load_dataset("crag-mm-2025/crag-mm-single-turn-public")

Generating validation split:   0%|          | 0/1938 [00:00<?, ? examples/s]

Generating public_test split:   0%|          | 0/1936 [00:00<?, ? examples/s]

### **ELIP**

In [10]:
from torch.utils.data import Dataset, DataLoader

# A. Create a simple Dataset Wrapper
class LlamaAlignmentDataset(Dataset):
    def __init__(self, hf_dataset, processor):
        self.data = hf_dataset
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image = item['image']
        prompt = item['turns']['query'][0]
        answer = item['answers']['ans_full'][0]

        return image, prompt, answer

valid_data = dataset['validation'].filter(lambda x: x['image'] is not None)

# pin_memory=True: Speeds up transfer to GPU
train_ds = LlamaAlignmentDataset(valid_data, processor)
train_loader = DataLoader(
    train_ds,
    batch_size=1,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    collate_fn=lambda x: x[0]
)

Filter:   0%|          | 0/1938 [00:00<?, ? examples/s]

In [11]:
import torch
import torch.nn as nn

class TextToVisionBridge(nn.Module):
    def __init__(self, base_model, num_extra_patches=10):
        super().__init__()
        self.model = base_model
        self.vision_model = base_model.vision_model
        self.language_model = base_model.language_model

        # Dynamically get device/dtype
        param = next(base_model.parameters())
        self.device = param.device
        self.dtype = param.dtype

        # --- Linear Bridge ---
        # Maps Text Dim (4096) -> Vision Dim (1280)
        self.text_to_patch = nn.Sequential(
            nn.Linear(4096, 5376),
            nn.GELU(),
            nn.Linear(5376, 6656),
            nn.GELU(),
            nn.Linear(6656, 9216),
            nn.GELU(),
            nn.Linear(9216, num_extra_patches * 1280)
        ).to(self.device, dtype=self.dtype)

        self.num_extra_patches = num_extra_patches

        # Freeze everything
        for p in self.vision_model.parameters(): p.requires_grad = False
        for p in self.language_model.parameters(): p.requires_grad = False

        # Unfreeze Bridge
        for p in self.text_to_patch.parameters(): p.requires_grad = True

    def forward(
        self,
        pixel_values: torch.Tensor,
        input_ids: torch.Tensor = None,
        attention_mask: torch.Tensor = None,
        aspect_ratio_ids: torch.Tensor = None,
        **kwargs,
    ) -> torch.Tensor:

        # Unpack and Flatten Images
        if pixel_values.dim() == 6:
            B, _, num_tiles, C, H, W = pixel_values.shape
        else:
            B, num_tiles, C, H, W = pixel_values.shape

        # Flatten Batch and Tiles together -> [B*Tiles, C, H, W]
        imgs = pixel_values.reshape(B * num_tiles, C, H, W)

        # 2. Get Text Summaries
        with torch.no_grad():
            text_emb = self.language_model.embed_tokens(input_ids)

        # Take last token state since it is decoder only model
        # [Batch_Size, Sequence_Length, Hidden_Dimension] -> [B, 4096]
        text_summary = text_emb[:, -1, :]

        # 3. Generate Extra Patches
        # Map to Vision Dim -> [B, Num_Patches, 1280]
        extra_patches = self.text_to_patch(text_summary).view(B, self.num_extra_patches, 1280)

        # Need [B * Tiles, Num_Patches, 1280]
        # [B, 1, N, D]
        extra_patches = extra_patches.unsqueeze(1)

        # Repeat for every tile -> [B, Tiles, N, D]
        # Duplicate the patches for every tile.
        extra_patches = extra_patches.repeat(1, num_tiles, 1, 1)

        # FLATTEN Batch and Tiles -> [B*Tiles, N, D]
        extra_patches = extra_patches.reshape(1, B * num_tiles, self.num_extra_patches, 1280)


        with torch.no_grad():
            # Convert raw pixels into patch embeddings
            x = self.vision_model.patch_embedding(imgs)

            # Flatten spatial dims (H*W) into Sequence length and transpose.
            # [Batch*Tiles, Seq_Len, 1280]
            x = x.flatten(2).transpose(1, 2)

            # Create a Learnable Class Token (CLS) initialized to zeros.
            cls_token = torch.zeros(x.shape[0], 1, x.shape[-1], device=x.device, dtype=x.dtype)

            # Append the CLS token to the sequence.
            x = torch.cat([cls_token, x], dim=1)

            # Apply Llama 3.2's special gated positional embeddings based on tile ID.
            x = self.vision_model.gated_positional_embedding(x, aspect_ratio_ids)

            # Apply Layer Normalization before the Transformer layers.
            x = self.vision_model.layernorm_pre(x)

        # Concat the extra patches
        x = torch.cat([extra_patches, x], dim=2)
        x = x.flatten(0, 1)


        # Run the Transformer Encoder Layers.
        x = self.vision_model.transformer(x)
        # Save all hidden states (used later for multi-stage feature aggregation).
        all_intermediate_hidden_states = x[1]
        x = x[0]


        with torch.no_grad():
            x = self.vision_model.layernorm_post(x)

        # Project vision features to language model dimension.
        vision_output = self.vision_model.global_transformer(x)

        # Get the indices of layers to extract features from (defined in model config). Llama extract special part of layer
        indices = self.model.config.vision_config.intermediate_layers_indices

        # Stack all layer outputs into one tensor.
        intermediate_hidden_states = torch.stack(all_intermediate_hidden_states, dim=-1)

        # Select only the specific layers required by the configuration.
        intermediate_hidden_states = intermediate_hidden_states[..., indices]

        # Combine intermediate features with the final vision output.
        output = torch.cat([intermediate_hidden_states, vision_output.last_hidden_state.unsqueeze(-1)], dim=-1)

        output = output.flatten(2)

        return output

In [12]:
import torch.nn.functional as F

def get_text_embedding(model, processor, text_list, device):
    """
    Encodes the 'Answer' text into a single vector using the LLM's embedding layer.
    We extract the last token embedding.
    """
    inputs = processor.tokenizer(
        text_list, return_tensors="pt", padding=True, truncation=True
    ).to(device)

    with torch.no_grad():
        embeddings = model.language_model.embed_tokens(inputs["input_ids"])

        # Extract the last token embedding
        last_embedding = embeddings[:, -1, :]

    return last_embedding

print("Starting Training...")
BATCH_SIZE = 1
GRAD_ACCUMULATION = 8
LEARNING_RATE = 2e-5
NUM_EPOCHS = 1
NUM_EXTRA_PATCHES = 10

bridge_model = TextToVisionBridge(model).to(model.device)
bridge_model.text_to_patch.train()
bridge_model.text_to_patch.to(torch.bfloat16)


optimizer = torch.optim.AdamW(bridge_model.text_to_patch.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=50
)
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    optimizer.zero_grad()

    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")

    for step, (image, prompt, answer) in enumerate(train_pbar):

        prompt_text = f"{prompt} <|end_of_text|>"
        text_inputs = processor.tokenizer(prompt_text, return_tensors="pt").to(model.device)


        image_inputs = processor(images=image, return_tensors="pt")
        image_inputs = {k: v.to(model.device) for k, v in image_inputs.items()}

        if "pixel_values" in image_inputs:
            image_inputs["pixel_values"] = image_inputs["pixel_values"].to(torch.bfloat16)

        # Output shape: [B, Seq_Len, Vision_Dim]
        vision_outputs = bridge_model(
            pixel_values=image_inputs["pixel_values"],
            input_ids=text_inputs["input_ids"],
            aspect_ratio_ids=image_inputs.get("aspect_ratio_ids", None)
        )

        # Project to llm space
        projected_vision = model.model.multi_modal_projector(vision_outputs)

        # takes every single vision patch
        # Treat every vision patch as an independent embedding. Row is patches column is embedding dimension
        vision_vec = projected_vision.view(-1, 4096)

        target_vec = get_text_embedding(model, processor, [prompt_text], model.device)

        v_norm = F.normalize(vision_vec, p=2, dim=1)
        t_norm = F.normalize(target_vec, p=2, dim=1)

        similarity_map = torch.matmul(v_norm, t_norm.t()).squeeze()

        k = 1500

        # Find the scores of the best K patches
        # Instead of getting one patch or whole image mean we extract a region of patch and calculate it mean.
        # So it more like letting the model have a higher score when they pay more attention of the object text mention
        top_k_scores, top_k_indices = torch.topk(similarity_map, k)

        loss = 1 - top_k_scores.mean()

        # Backward
        loss = loss / GRAD_ACCUMULATION
        loss.backward()

        if (step + 1) % GRAD_ACCUMULATION == 0:
          optimizer.step()
          optimizer.zero_grad()

          # Update Scheduler based on current loss
          current_loss_val = loss.item() * GRAD_ACCUMULATION
          scheduler.step(current_loss_val)

          print(f"Epoch {epoch} | Step {step} | Loss: {current_loss_val:.4f}")

print("Training Complete.")

Starting Training...


Epoch 1:   1%|          | 8/1423 [00:07<13:19,  1.77it/s]

Epoch 0 | Step 7 | Loss: 0.9844


Epoch 1:   1%|          | 16/1423 [00:10<10:39,  2.20it/s]

Epoch 0 | Step 15 | Loss: 0.9883


...

Epoch 0 | Step 1407 | Loss: 0.9219


Epoch 1: 100%|█████████▉| 1416/1423 [10:20<00:03,  2.30it/s]

Epoch 0 | Step 1415 | Loss: 0.9258


Epoch 1: 100%|██████████| 1423/1423 [10:24<00:00,  2.28it/s]

Training Complete.


In [16]:
example = dataset['public_test'][0]
image = example['image']
prompt = example['turns']['query'][0]

bridge_model.eval()
inputs = processor(image, prompt, return_tensors="pt").to(model.device)
inputs = {
    k: (v.to(torch.bfloat16) if torch.is_floating_point(v) else v)
    for k, v in inputs.items()
}

out = bridge_model(**inputs)
result = model.model.multi_modal_projector(out)

batch_size = 1
num_images = 1
num_tiles = 4

# Reshape: [4, 1611, 4096] -> [1, 1, 4, 1611, 4096]
cross_attention_states = result.view(batch_size, num_images, num_tiles, 1611, 4096)

cross_attention_mask = torch.ones(
    (batch_size, num_images, num_tiles, 1611),
    dtype=torch.bfloat16,
    device=result.device
)
cross_attention_mask = cross_attention_mask.view(batch_size, 1, 1, -1)

with torch.no_grad():
    outputs = model.language_model(
    input_ids=inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    cross_attention_states=cross_attention_states,
    cross_attention_mask=cross_attention_mask
)


input_ids = inputs['input_ids']
generated_text = ""

print("Generating...")

for i in range(150):
    outputs = model.language_model(
        input_ids=input_ids,
        cross_attention_states=cross_attention_states,
        cross_attention_mask=cross_attention_mask
    )

    logits = model.lm_head(outputs.last_hidden_state)

    # Get the next token ID
    next_token_logits = logits[:, -1, :]
    next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(-1)

    # 4. Check for EOS (End of Sentence)
    if next_token_id.item() == processor.tokenizer.eos_token_id:
        print("\n<EOS reached>")
        break

    input_ids = torch.cat([input_ids, next_token_id], dim=1)

    # Decode and print
    word = processor.tokenizer.decode(next_token_id[0])
    generated_text += word
    print(word, end="", flush=True)

Generating...
The vehicle's model is a 1969-1970 Chevrolet Camaro, and the engine is a 350. The 1969-1970 Chevrolet Camaro with a 350 engine has a top speed of approximately 120 mph. The 1969-1970 Chevrolet Camaro with a 350 engine has a top speed of approximately 120 mph. The 1969-1970 Chevrolet Camaro with a 350 engine has a top speed of approximately 120 mph. The 1969-1970 Chevrolet Camaro with a 350 engine has a top speed of approximately 120 mph. The 1969-1970 Chevrolet Camaro with a 350 engine has a top speed of approximately 120 mph. The